In [1]:
import pandas as pd
import os
import pyarrow.parquet as pq
from tqdm import tqdm
import seaborn as sns
import matplotlib.pyplot as plt
import torch
import torch.nn as nn
from torchvision.models import resnet50, resnet18

In [2]:
# def get_batch(batchfile):
#     batch1 = pq.ParquetFile(batchfile)
#     it = batch1.iter_batches()
#     batch1 = next(it).to_pandas()
#     temp = batchfile.split('/')[-1].split('batch_')[1].split('.')[0]
#     batch1['Batch'] = temp
#     return(batch1)
#
# def get_pq(pqfile):
#     pq_df = pq.ParquetFile(pqfile)
#     it = pq_df.iter_batches()
#     pq_df = next(it).to_pandas()
#     return(pq_df)

In [3]:
sensor = pd.read_csv('/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/sensor_geometry.csv')
train_meta = pq.ParquetFile('/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/train_meta.parquet')
# # train_meta = pq.ParquetFile("/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/train/batch_1.parquet")
# it = train_meta.iter_batches()
# train_meta = next(it).to_pandas()

In [4]:
# train_meta.head(20)

In [5]:
# i = 1
# while True:
#     try:
#         item = next(it)
#         i += 1
#     except StopIteration:
#         break
#
# print(i)

In [6]:
# print(train_meta)

In [7]:
# path_batch = '/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/train/'
# batchfile = path_batch + 'batch_1.parquet'
# batch1 = pq.ParquetFile(batchfile)
# it = batch1.iter_batches()
# batch1 = next(it).to_pandas()
# batch1.head(100)

In [8]:
# path_batch = '/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/train/'
# sensor_info = [get_batch(path_batch+'batch_' + str(i+1) + '.parquet') for i in tqdm(range(2))]
# sensor_info_df = pd.concat(sensor_info).reset_index()

In [9]:
# sensor_info_df.head()

In [10]:
# plt.figure(figsize = (20,10))
# sns.set(font_scale = 1,style = 'ticks')
# for i in range(16):
#     plt.subplot(4,4,i+1)
#     df = sensor_info_df[(sensor_info_df.event_id==sensor_info_df.event_id.unique()[i])]
#     sns.lineplot(data = df,x = 'time',y = 'charge',hue = 'auxiliary',)
#     plt.title('Batch 1, Event ID'+str(sensor_info_df.event_id.unique()[i]))
#     plt.legend(loc = 'upper right')
#     sns.despine()
# plt.tight_layout()

In [11]:
from Resnet import resnet101
import torch.nn as nn
model = resnet101(pretrained = False)
model.fc = nn.Sequential(nn.ReLU(),nn.Linear(in_features=2048, out_features=2)) # Changed FC layer for our task

In [12]:
# import torch.nn as nn
# model = resnet34(pretrained = False)
# model.fc = nn.Sequential(nn.ReLU(),nn.Linear(in_features=512, out_features=2)) # Changed FC layer for our task
# # model = model.to(device)

In [13]:
# from CustomConvolution import CustomConvolution
# model = CustomConvolution()
# # model = model.to(device)

In [14]:
from sklearn.metrics import mean_squared_error
import numpy as np

def evaluation(dataloader):
    predictions = torch.tensor([], dtype=torch.float).to(device) # Tensor for prediction value appending
    actual = torch.tensor([], dtype=torch.float).to(device) # Tensor for answer value appending
    with torch.no_grad():
        model.eval()
        for data in dataloader:
            inputs, values = data['input_tensor'].float().to(device),data['label'].to(device)
            outputs = model(inputs).to(device)
            predictions = torch.cat((predictions, torch.stack([torch.argmax(o) for o in outputs])),0)
            actual = torch.cat((actual, values), 0)
    predictions = predictions.cpu().numpy()
    actual = actual.cpu().numpy()
    rmse = np.sqrt(mean_squared_error(predictions, actual))
    return rmse

In [15]:
from torch import optim
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'


pqfile = '/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/train_meta.parquet'
path_batch = '/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/train_2/'

batch_num=1 # There are 660 batches total, and the batch number should be iterated in range(660).
lr = 1e-06
num_epochs = 1
batch_size = 4

model = model.to(device)
optimizer = optim.Adam(model.parameters(), lr=lr)
# loss_function = nn.BCEWithLogitsLoss().to(device)
loss_function = nn.MSELoss().to(device)

In [16]:
from torch.utils.data import DataLoader
from Icecube_Dataloader import ICECUBE_Dataset, collate_fn

ice_dataset = ICECUBE_Dataset(pqfile,path_batch,batch_num)
train_dataset = ice_dataset
# proportions = [.75, .10, .15]
# lengths = [int(p * len(ice_dataset)) for p in proportions]
# lengths[-1] = len(ice_dataset) - sum(lengths[:-1])
# train_dataset, val_dataset, test_dataset = torch.utils.data.random_split(ice_dataset, lengths)

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True,collate_fn=collate_fn, num_workers=11)
# val_dataloader = DataLoader(val_dataset, batch_size=batch_size, shuffle=True,collate_fn=collate_fn, num_workers=11)
# test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=True,collate_fn=collate_fn, num_workers=11)

In [17]:
print(device)

cpu


In [18]:
# model(ice_dataset.__getitem__(0)['input_tensor'].to(device))

In [19]:
# print(ice_dataset.__getitem__(0)['input_tensor'].shape)
# for i in range(20):
#     print(ice_dataset.__getitem__(i)['input_tensor'].shape)

In [20]:
# train_dataloader.__getitem__(0)
# # temp = iter(train_dataloader)
# # next(temp)

In [21]:
params = {
    'num_epochs':num_epochs,
    'optimizer':optimizer,
    'loss_function':loss_function,
    'train_dataloader':train_dataloader,
    # 'val_dataloader': val_dataloader,
    # 'test_dataloader': test_dataloader,
    'device':device,
    'num_epoch' : num_epochs
}


In [22]:
# train_dataset[0]['input_tensor'].shape

In [23]:
# train_dataset[0]['label'] # This would be 'azimuth','zenith'

In [24]:
import gc
train_losses = []

def train(model, params):
    model.train()
    loss_function=params["loss_function"]
    train_dataloader=params["train_dataloader"]
    # val_dataloader=params["val_dataloader"]
    # test_dataloader=params["test_dataloader"]

    # device=params["device"]
    # for epoch in range(0, num_epochs):
    #     with tqdm(train_dataloader,unit = 'batch') as tepoch:
    #         for dat in train_dataloader:
    #             tepoch.set_description(f"Epoch {epoch}")
    #             inputs, labels = dat['input_tensor'].to(device),dat['label'].to(device)
    #             optimizer.zero_grad()
    #             outputs = model(inputs).to(device)
    #             train_loss = loss_function(outputs.float(),labels.float())
    #             train_loss = train_loss.requires_grad_(True)
    #             train_loss.backward()
    #             optimizer.step()
    #             tepoch.set_postfix(loss=train_loss.item())

    aa = 0

    device=params["device"]
    for epoch in range(0, num_epochs):
        for dat in tqdm(train_dataloader):
            inputs, labels = dat['input_tensor'].to(device),dat['label'].to(device)
            optimizer.zero_grad()
            outputs = model(inputs).to(device)
            train_loss = loss_function(outputs.float(),labels.float())
            if aa%10 == 0:
                train_losses.append(train_loss.item())
            # if aa%100==0:
            #     print(f"Train loss: {train_loss.item()}")
            #     gc.collect()
            # train_loss = train_loss.requires_grad_(True)
            train_loss.backward()
            optimizer.step()
            # torch.cuda.empty_cache()
            aa = aa+1

    model.eval()
    # train_rmse = evaluation(train_dataloader)
    # val_rmse = evaluation(val_dataloader)

    # print(f"Train Loss: {train_rmse}")
    # print(" Train Loss: %.4f, Validation Loss: %.4f" %(train_rmse, val_rmse))
    # torch.cuda.empty_cache()
    # gc.collect()
    return 0


train(model, params)

100%|██████████| 50000/50000 [2:31:36<00:00,  5.50it/s]   


0

In [25]:
torch.save(model.state_dict(), "./model_101.pth")

In [26]:
import pickle

# Save to file
with open('losses.pickle', 'wb') as f:
    pickle.dump(train_losses, f)

In [27]:
# torch.save(model, "./model_all.pth")

In [28]:
# pqfile = '/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/test_meta.parquet'
# path_batch = '/home/washindeiru/primaryDrive/iceCube/icecube-neutrinos-in-deep-ice/test/'
# batch_num=661
# inference_dataset = ICECUDE_Dataset(pqfile,path_batch,batch_num,'test')
# inference_dataloader = DataLoader(inference_dataset, batch_size=1)

In [29]:
# model.eval()
# output_lst=[]
# for dat in inference_dataloader:
#     inputs = dat['input_tensor'].to(device)
#     outputs = model(inputs).to(device)
#     outputs = outputs.cpu().detach().squeeze().numpy()
#     outputs = outputs.tolist()
#     output_lst.append(outputs)

In [30]:
# sample_sub = pq.read_table('/kaggle/input/icecube-neutrinos-in-deep-ice/sample_submission.parquet').to_pandas()
# batch661 = pq.read_table('/kaggle/input/icecube-neutrinos-in-deep-ice/test/batch_661.parquet').to_pandas()


In [31]:
# sample_sub

In [32]:
# submission = pd.DataFrame(output_lst)
# submission.index = batch661.index.unique().tolist()
# submission.reset_index(inplace = True)
# submission.columns = ['event_id','azimuth','zenith']

In [33]:
# submission